In [34]:
import pandas as pd
import re
import os
from dotenv import main
from sqlalchemy import create_engine, text, String, Numeric, Date
from datetime import datetime

In [35]:
# Definindo a função para adicionar zero à esquerda
def add_zero(value):
    if value in ['100', '120', '300']:
        return f'0{value}'
    return value

In [36]:
main.load_dotenv(dotenv_path="../../Env/.env", override= True)

True

In [37]:
financeiro = create_engine(os.getenv("DB_DIALECT") + "financeiro")
stage = create_engine(os.getenv("DB_DIALECT") + "stage")

In [38]:
caminho_arquivo = "base/forn_act_bgt_24.xlsx"

colunas = [
    '      Valor/MR',
    'Nº doc.ref',
    'Conta',
    'Centro cst',
    'Empr',
    'Dt.lçto.',
    'Data doc.',
    'Texto',
    'Atribuição',
    'LW Forn',
    'Fornecedor',
    'Ano/Mês',
    'BGT / ACT',
    'REF+Nome',
    'REF2+Nome'
]

tipo_de_dados = {
    '      Valor/MR' : float,
    'Nº doc.ref' : str,
    'Conta' : str,
    'Centro cst' : str,
    'Empr' : str,
    'Dt.lçto.' : str,
    'Data doc.' : str,
    'Texto' : str,
    'Atribuição' : str,
    'LW Forn' : str,
    'Fornecedor' : str,
    'Ano/Mês' : str,
    'BGT / ACT' : str,
    'REF+Nome' : str,
    'REF2+Nome' : str
}

In [40]:
df = pd.read_excel(caminho_arquivo, sheet_name = "Planilha1", dtype = tipo_de_dados)

In [41]:
nome_colunas = ['valor', 'num_doc_ref',	'cod_conta_contabil', 'cod_centro_custo', 'cod_empresa', 'dt_lancamento', 'dt_documento',	'texto',	'atribuicao',	'cod_fornecedor',	'desc_fornecedor',	'dt_referencia', 'tipo',	'ref_nome',	'ref2_nome']

In [42]:
df2 = df

In [ ]:
df = df2

In [ ]:
# ## Filtro
# df = df[(df["ref_nome"] == "R.C - Operation ") | (df["ref_nome"] == "R.I - Marketing")]

In [43]:
# Removendo nulos
df = df[(~df[df.columns[0]].isna()) & (df[df.columns[0]] != "nan")]

In [44]:
# Selecionando apenas as colunas necessárias
df = df[[
    df.columns[0], 
    df.columns[1],
    df.columns[2],
    df.columns[3],
    df.columns[4],
    df.columns[5],
    df.columns[6],
    df.columns[7],
    df.columns[8],
    df.columns[9],
    df.columns[10],
    df.columns[11],
    df.columns[14],
    df.columns[18],
    df.columns[19]
    ]]

In [45]:
df = df.rename(columns = dict(zip(df.columns, nome_colunas)))

In [46]:
df["dt_lancamento"] = pd.to_datetime(df["dt_lancamento"], format='%d.%m.%Y')
df["dt_documento"] = pd.to_datetime(df["dt_documento"], format='%d.%m.%Y')


# Ajustando o tipo de dt_referencia
df["dt_referencia"] = df["dt_referencia"].astype('str')
df.loc[df["dt_referencia"] == "2023.1", "dt_referencia"] = "2023.10"
df.loc[df["dt_referencia"] == "2024.1", "dt_referencia"] = "2024.10"

df["dt_referencia"] = pd.to_datetime(df["dt_referencia"], format='%Y.%m', dayfirst = True)

# Corrigindo tipo
df.loc[df["tipo"] == "ACT 2023", "tipo"] = "act"
df.loc[df["tipo"] == "BGT 2023", "tipo"] = "bgt"
df.loc[df["tipo"] == "ACT 2024", "tipo"] = "act"
df.loc[df["tipo"] == "BGT 2024", "tipo"] = "bgt"

# Corrigindo arredondamento
df["valor"] = df["valor"].round(2)

# Adicionando 0 a esquerda das empresas 100, 120, 300
df['cod_empresa'] = df['cod_empresa'].apply(add_zero)

In [ ]:
# df = df[df["dt_referencia"] == "2024-05-01"]

In [47]:
df.to_sql("custo_fornecedores_upsert", stage, if_exists = "replace", index = False, 
                            dtype = {
                                    'valor' : Numeric(15,2), 
                                    'num_doc_ref' : String,
                                    'cod_conta_contabil' : String,
                                    'cod_centro_custo' : String,
                                    'cod_empresa' : String,
                                    'dt_lancamento' : Date,
                                    'dt_documento' : Date,
                                    'texto' : String,
                                    'atribuicao' : String,
                                    'cod_fornecedor' : String,
                                    'desc_fornecedor' : String,
                                    'dt_referencia' : Date,
                                    'tipo' : String,
                                    'ref_nome' : String,
                                    'ref2_nome' : String
                            })

104